In [96]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import ast
print(os.listdir())

['gensimtutorial.py', 'biz_2star.csv', 'biz_3star.csv', 'first_buz.csv', '.DS_Store', 'Data Preparation.ipynb', 'Archive', '100buz.csv', 'Untitled.ipynb', 'EDA pt 2.ipynb', 'stats100.csv', 'dirichlet_plots.png', '10buz.csv', 'stats.csv', 'dirichletplot.py', 'LDAVisual.py', 'Yelper.py', '__pycache__', 'doc_topic.csv', 'stats500.csv', 'FINALL_doc_topic.csv', 'BusinessKeywords.csv', 'Final EDA Pt 1.ipynb', '1000buz.csv', 'LDA.png', 'yelp_dataset', 'sample_buz.csv', 'topic_mixture.csv', 'Lab.ipynb', 'PreEDA.ipynb', 'KNN Review2.ipynb', '.ipynb_checkpoints', 'LDAVisual 2.py', 'LDAFINALTEST.ipynb', 'Final training.ipynb', 'Lab 1 Project.ipynb', 'KMeaned.csv', 'LDAVisual.py.zip', 'n-gram and pre-processing lab.ipynb', '500buz.csv', 'KMeans.ipynb', 'Final Data Collection.ipynb', 'simpleLDA.py']


In [14]:
df = pd.read_csv('FINALL_doc_topic.csv')
df_by_buz = df.sort_values(by=['business_id'])
df_b_all = df_by_buz.reset_index()
df_b_all.drop(['index','Unnamed: 0'],axis = 1, inplace = True)
df_b_100 = df_b_all[:100]

In [15]:
df_biz = pd.read_csv('yelp_dataset/business.csv')

In [16]:
df_biz.head()

address  \
0     2818 E Camino Acequia Drive   
1            30 Eglinton Avenue W   
2       10110 Johnston Rd, Ste 15   
3   15655 W Roosevelt St, Ste 237   
4  4209 Stuart Andrew Blvd, Ste F   

                                          attributes             business_id  \
0                           {'GoodForKids': 'False'}  1SWheh84yJXfytovILXOAQ   
1  {'RestaurantsReservations': 'True', 'GoodForMe...  QXAEGFB4oINsVuTFxEYKFQ   
2  {'GoodForKids': 'True', 'NoiseLevel': "u'avera...  gnKjwL_1w79qoiV3IC_xQQ   
3                                                NaN  xvX2CttrVhyG2z1dFg_0xw   
4  {'BusinessAcceptsBitcoin': 'False', 'ByAppoint...  HhyxOkGAM07SRYtlQ4wMFQ   

                                          categories         city  \
0                                  Golf, Active Life      Phoenix   
1  Specialty Food, Restaurants, Dim Sum, Imported...  Mississauga   
2                  Sushi Bars, Restaurants, Japanese    Charlotte   
3                      Insurance, Financial Services     Goodyear   
4  Plumbing, Shopping, Local Services, Home Servi...    Charlotte   

                                               hours  is_open   latitude  \
0                                                NaN        0  33.522143   
1  {'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...        1  43.605499   
2  {'Monday': '17:30-21:30', 'Wednesday': '17:30-...        1  35.092564   
3  {'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...        1  33.455613   
4  {'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ...        1  35.190012   

    longitude                             name postal_code  review_count  \
0 -112.018481       Arizona Biltmore Golf Club       85016             5   
1  -79.652289       Emerald Chinese Restaurant     L5R 3E7           128   
2  -80.859132      Musashi Japanese Restaurant       28210           170   
3 -112.395596  Farmers Insurance - Paul Lorenz       85338             3   
4  -80.887223              Queen City Plumbing       28217             4   

   stars state  
0    3.0    AZ  
1    2.5    ON  
2    4.0    NC  
3    5.0    AZ  
4    4.0    NC

In [17]:
df_biz_list = list(df_b_all['business_id'])
len(df_biz_list)

30000

In [68]:
# Identify the type of business please!!!!
cat_list = []
name_list = []
count = 0
for biz_id in df_biz_list:
    print(count)
    category = df_biz['categories'].loc[(df_biz['business_id'] == biz_id)]
    category_nodigit = ''.join([i for i in category if not i.isdigit()])
    name = df_biz['name'].loc[(df_biz['business_id'] == biz_id)]
    name_nodigit = ''.join([i for i in name if not i.isdigit()])
    cat_list.append(category_nodigit)
    name_list.append(name_nodigit)
    count += 1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061


3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705


5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347


6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991


...

23816
23817
23818
23819
23820
23821
23822
23823
23824
23825
23826
23827
23828
23829
23830
23831
23832
23833
23834
23835
23836
23837
23838
23839
23840
23841
23842
23843
23844
23845
23846
23847
23848
23849
23850
23851
23852
23853
23854
23855
23856
23857
23858
23859
23860
23861
23862
23863
23864
23865
23866
23867
23868
23869
23870
23871
23872
23873
23874
23875
23876
23877
23878
23879
23880
23881
23882
23883
23884
23885
23886
23887
23888
23889
23890
23891
23892
23893
23894
23895
23896
23897
23898
23899
23900
23901
23902
23903
23904
23905
23906
23907
23908
23909
23910
23911
23912
23913
23914
23915
23916
23917
23918
23919
23920
23921
23922
23923
23924
23925
23926
23927
23928
23929
23930
23931
23932
23933
23934
23935
23936
23937
23938
23939
23940
23941
23942
23943
23944
23945
23946
23947
23948
23949
23950
23951
23952
23953
23954
23955
23956
23957
23958
23959
23960
23961
23962
23963
23964
23965
23966
23967
23968
23969
23970
23971
23972
23973
23974
23975
23976
23977
23978
23979
23980
23981
2398

25182
25183
25184
25185
25186
25187
25188
25189
25190
25191
25192
25193
25194
25195
25196
25197
25198
25199
25200
25201
25202
25203
25204
25205
25206
25207
25208
25209
25210
25211
25212
25213
25214
25215
25216
25217
25218
25219
25220
25221
25222
25223
25224
25225
25226
25227
25228
25229
25230
25231
25232
25233
25234
25235
25236
25237
25238
25239
25240
25241
25242
25243
25244
25245
25246
25247
25248
25249
25250
25251
25252
25253
25254
25255
25256
25257
25258
25259
25260
25261
25262
25263
25264
25265
25266
25267
25268
25269
25270
25271
25272
25273
25274
25275
25276
25277
25278
25279
25280
25281
25282
25283
25284
25285
25286
25287
25288
25289
25290
25291
25292
25293
25294
25295
25296
25297
25298
25299
25300
25301
25302
25303
25304
25305
25306
25307
25308
25309
25310
25311
25312
25313
25314
25315
25316
25317
25318
25319
25320
25321
25322
25323
25324
25325
25326
25327
25328
25329
25330
25331
25332
25333
25334
25335
25336
25337
25338
25339
25340
25341
25342
25343
25344
25345
25346
25347
2534

26558
26559
26560
26561
26562
26563
26564
26565
26566
26567
26568
26569
26570
26571
26572
26573
26574
26575
26576
26577
26578
26579
26580
26581
26582
26583
26584
26585
26586
26587
26588
26589
26590
26591
26592
26593
26594
26595
26596
26597
26598
26599
26600
26601
26602
26603
26604
26605
26606
26607
26608
26609
26610
26611
26612
26613
26614
26615
26616
26617
26618
26619
26620
26621
26622
26623
26624
26625
26626
26627
26628
26629
26630
26631
26632
26633
26634
26635
26636
26637
26638
26639
26640
26641
26642
26643
26644
26645
26646
26647
26648
26649
26650
26651
26652
26653
26654
26655
26656
26657
26658
26659
26660
26661
26662
26663
26664
26665
26666
26667
26668
26669
26670
26671
26672
26673
26674
26675
26676
26677
26678
26679
26680
26681
26682
26683
26684
26685
26686
26687
26688
26689
26690
26691
26692
26693
26694
26695
26696
26697
26698
26699
26700
26701
26702
26703
26704
26705
26706
26707
26708
26709
26710
26711
26712
26713
26714
26715
26716
26717
26718
26719
26720
26721
26722
26723
2672

27938
27939
27940
27941
27942
27943
27944
27945
27946
27947
27948
27949
27950
27951
27952
27953
27954
27955
27956
27957
27958
27959
27960
27961
27962
27963
27964
27965
27966
27967
27968
27969
27970
27971
27972
27973
27974
27975
27976
27977
27978
27979
27980
27981
27982
27983
27984
27985
27986
27987
27988
27989
27990
27991
27992
27993
27994
27995
27996
27997
27998
27999
28000
28001
28002
28003
28004
28005
28006
28007
28008
28009
28010
28011
28012
28013
28014
28015
28016
28017
28018
28019
28020
28021
28022
28023
28024
28025
28026
28027
28028
28029
28030
28031
28032
28033
28034
28035
28036
28037
28038
28039
28040
28041
28042
28043
28044
28045
28046
28047
28048
28049
28050
28051
28052
28053
28054
28055
28056
28057
28058
28059
28060
28061
28062
28063
28064
28065
28066
28067
28068
28069
28070
28071
28072
28073
28074
28075
28076
28077
28078
28079
28080
28081
28082
28083
28084
28085
28086
28087
28088
28089
28090
28091
28092
28093
28094
28095
28096
28097
28098
28099
28100
28101
28102
28103
2810

29307
29308
29309
29310
29311
29312
29313
29314
29315
29316
29317
29318
29319
29320
29321
29322
29323
29324
29325
29326
29327
29328
29329
29330
29331
29332
29333
29334
29335
29336
29337
29338
29339
29340
29341
29342
29343
29344
29345
29346
29347
29348
29349
29350
29351
29352
29353
29354
29355
29356
29357
29358
29359
29360
29361
29362
29363
29364
29365
29366
29367
29368
29369
29370
29371
29372
29373
29374
29375
29376
29377
29378
29379
29380
29381
29382
29383
29384
29385
29386
29387
29388
29389
29390
29391
29392
29393
29394
29395
29396
29397
29398
29399
29400
29401
29402
29403
29404
29405
29406
29407
29408
29409
29410
29411
29412
29413
29414
29415
29416
29417
29418
29419
29420
29421
29422
29423
29424
29425
29426
29427
29428
29429
29430
29431
29432
29433
29434
29435
29436
29437
29438
29439
29440
29441
29442
29443
29444
29445
29446
29447
29448
29449
29450
29451
29452
29453
29454
29455
29456
29457
29458
29459
29460
29461
29462
29463
29464
29465
29466
29467
29468
29469
29470
29471
29472
2947

In [70]:
df_b_all['category'] = cat_list
df_b_all['name'] = name_list
df_b_all

business_id  \
0      0FUtlsQrJI7LhqDPxLumEw   
1      0FUtlsQrJI7LhqDPxLumEw   
2      0FUtlsQrJI7LhqDPxLumEw   
3      0FUtlsQrJI7LhqDPxLumEw   
4      0FUtlsQrJI7LhqDPxLumEw   
5      0FUtlsQrJI7LhqDPxLumEw   
6      0FUtlsQrJI7LhqDPxLumEw   
7      0FUtlsQrJI7LhqDPxLumEw   
8      0FUtlsQrJI7LhqDPxLumEw   
9      0FUtlsQrJI7LhqDPxLumEw   
10     0FUtlsQrJI7LhqDPxLumEw   
11     0FUtlsQrJI7LhqDPxLumEw   
12     0FUtlsQrJI7LhqDPxLumEw   
13     0FUtlsQrJI7LhqDPxLumEw   
14     0FUtlsQrJI7LhqDPxLumEw   
15     0FUtlsQrJI7LhqDPxLumEw   
16     0FUtlsQrJI7LhqDPxLumEw   
17     0FUtlsQrJI7LhqDPxLumEw   
18     0FUtlsQrJI7LhqDPxLumEw   
19     0FUtlsQrJI7LhqDPxLumEw   
20     0FUtlsQrJI7LhqDPxLumEw   
21     0FUtlsQrJI7LhqDPxLumEw   
22     0FUtlsQrJI7LhqDPxLumEw   
23     0FUtlsQrJI7LhqDPxLumEw   
24     0FUtlsQrJI7LhqDPxLumEw   
25     0FUtlsQrJI7LhqDPxLumEw   
26     0FUtlsQrJI7LhqDPxLumEw   
27     0FUtlsQrJI7LhqDPxLumEw   
28     0FUtlsQrJI7LhqDPxLumEw   
29     0FUtlsQrJI7LhqDPxLumEw   
...                       ...   
29970  yfxDa8RFOvJPQh0rNtakHA   
29971  yfxDa8RFOvJPQh0rNtakHA   
29972  yfxDa8RFOvJPQh0rNtakHA   
29973  yfxDa8RFOvJPQh0rNtakHA   
29974  yfxDa8RFOvJPQh0rNtakHA   
29975  yfxDa8RFOvJPQh0rNtakHA   
29976  yfxDa8RFOvJPQh0rNtakHA   
29977  yfxDa8RFOvJPQh0rNtakHA   
29978  yfxDa8RFOvJPQh0rNtakHA   
29979  yfxDa8RFOvJPQh0rNtakHA   
29980  yfxDa8RFOvJPQh0rNtakHA   
29981  yfxDa8RFOvJPQh0rNtakHA   
29982  yfxDa8RFOvJPQh0rNtakHA   
29983  yfxDa8RFOvJPQh0rNtakHA   
29984  yfxDa8RFOvJPQh0rNtakHA   
29985  yfxDa8RFOvJPQh0rNtakHA   
29986  yfxDa8RFOvJPQh0rNtakHA   
29987  yfxDa8RFOvJPQh0rNtakHA   
29988  yfxDa8RFOvJPQh0rNtakHA   
29989  yfxDa8RFOvJPQh0rNtakHA   
29990  yfxDa8RFOvJPQh0rNtakHA   
29991  yfxDa8RFOvJPQh0rNtakHA   
29992  yfxDa8RFOvJPQh0rNtakHA   
29993  yfxDa8RFOvJPQh0rNtakHA   
29994  yfxDa8RFOvJPQh0rNtakHA   
29995  yfxDa8RFOvJPQh0rNtakHA   
29996  yfxDa8RFOvJPQh0rNtakHA   
29997  yfxDa8RFOvJPQh0rNtakHA   
29998  yfxDa8RFOvJPQh0rNtakHA   
29999  yfxDa8RFOvJPQh0rNtakHA   

                                                  review  star   topic 0  \
0      We weren't a fan of this place.  Food was just...   2.0  0.000000   
1      The food was good just like it is at Joes BBQ....   2.0  0.000000   
2      I was very excited to visit this place after r...   2.0  0.058923   
3      Prices were high and the food is nothing speci...   2.0  0.000000   
4      $17 for a sandwich and fries that I had to wai...   1.0  0.000000   
5      We are new to the area and heard great things ...   2.0  0.000000   
6      The cashier refused to take our lunch order wh...   1.0  0.000000   
7      Was really looking forward to this well known ...   1.0  0.000000   
8      We have been coming to this place for several ...   2.0  0.000000   
9      The first time I went here, I got a bbq hambur...   2.0  0.000000   
10     The atmosphere is gorgeous and I loved the gar...   2.0  0.000000   
11     We decided to have dinner here because they ha...   2.0  0.000000   
12     Not sure what the big deal is. It's regular fa...   2.0  0.000000   
13     Found Joe's Farm on the list of places visited...   2.0  0.000000   
14     It's been several years since I've been to the...   2.0  0.000000   
15     I love Joes Farm Grill. The food is amazing an...   2.0  0.000000   
16     Usually a great place to eat, last time around...   1.0  0.084681   
17     Something happened with the new upgrade...im n...   2.0  0.000000   
18     came here tonight and was not impressed at all...   2.0  0.000000   
19     Pros: Cool place with a very unique atmosphere...   2.0  0.000000   
20     I was looking forward to going here as I've he...   2.0  0.000000   
21     Went today. Not very busy and still took about...   1.0  0.000000   
22     I don't know why so many are crazy for this di...   2.0  0.000000   
23     First, paper plates , second, there are kids p...   2.0  0.000000   
24     Once again come on Guy you must be eating diff...   2.0  0.00000

In [33]:
# df.reindex(sorted(df.columns), axis=1)

In [113]:
sns.set(font_scale=1.5)
plt.subplots(figsize=(15,15)) 
ax = sns.heatmap(df_b_all.drop(['business_id','name','category','review','star'],axis = 1),cmap = 'inferno')

In [133]:
plt.subplots(figsize=(10,10)) 
ax = sns.heatmap(df_b_all[7135:7304].drop(['business_id','name','category','review','star'],axis = 1), cbar = False)

In [134]:
 df_b_all[7135:7304]

business_id  \
7135  Cni2l-VKG_pdospJ6xliXQ   
7136  Cni2l-VKG_pdospJ6xliXQ   
7137  Cni2l-VKG_pdospJ6xliXQ   
7138  Cni2l-VKG_pdospJ6xliXQ   
7139  Cni2l-VKG_pdospJ6xliXQ   
7140  Cni2l-VKG_pdospJ6xliXQ   
7141  Cni2l-VKG_pdospJ6xliXQ   
7142  Cni2l-VKG_pdospJ6xliXQ   
7143  Cni2l-VKG_pdospJ6xliXQ   
7144  Cni2l-VKG_pdospJ6xliXQ   
7145  Cni2l-VKG_pdospJ6xliXQ   
7146  Cni2l-VKG_pdospJ6xliXQ   
7147  Cni2l-VKG_pdospJ6xliXQ   
7148  Cni2l-VKG_pdospJ6xliXQ   
7149  Cni2l-VKG_pdospJ6xliXQ   
7150  Cni2l-VKG_pdospJ6xliXQ   
7151  Cni2l-VKG_pdospJ6xliXQ   
7152  Cni2l-VKG_pdospJ6xliXQ   
7153  Cni2l-VKG_pdospJ6xliXQ   
7154  Cni2l-VKG_pdospJ6xliXQ   
7155  Cni2l-VKG_pdospJ6xliXQ   
7156  Cni2l-VKG_pdospJ6xliXQ   
7157  Cni2l-VKG_pdospJ6xliXQ   
7158  Cni2l-VKG_pdospJ6xliXQ   
7159  Cni2l-VKG_pdospJ6xliXQ   
7160  Cni2l-VKG_pdospJ6xliXQ   
7161  Cni2l-VKG_pdospJ6xliXQ   
7162  Cni2l-VKG_pdospJ6xliXQ   
7163  Cni2l-VKG_pdospJ6xliXQ   
7164  Cni2l-VKG_pdospJ6xliXQ   
...                      ...   
7274  Cni2l-VKG_pdospJ6xliXQ   
7275  Cni2l-VKG_pdospJ6xliXQ   
7276  Cni2l-VKG_pdospJ6xliXQ   
7277  Cni2l-VKG_pdospJ6xliXQ   
7278  Cni2l-VKG_pdospJ6xliXQ   
7279  Cni2l-VKG_pdospJ6xliXQ   
7280  Cni2l-VKG_pdospJ6xliXQ   
7281  Cni2l-VKG_pdospJ6xliXQ   
7282  Cni2l-VKG_pdospJ6xliXQ   
7283  Cni2l-VKG_pdospJ6xliXQ   
7284  Cni2l-VKG_pdospJ6xliXQ   
7285  Cni2l-VKG_pdospJ6xliXQ   
7286  Cni2l-VKG_pdospJ6xliXQ   
7287  Cni2l-VKG_pdospJ6xliXQ   
7288  Cni2l-VKG_pdospJ6xliXQ   
7289  Cni2l-VKG_pdospJ6xliXQ   
7290  Cni2l-VKG_pdospJ6xliXQ   
7291  Cni2l-VKG_pdospJ6xliXQ   
7292  Cni2l-VKG_pdospJ6xliXQ   
7293  Cni2l-VKG_pdospJ6xliXQ   
7294  Cni2l-VKG_pdospJ6xliXQ   
7295  Cni2l-VKG_pdospJ6xliXQ   
7296  Cni2l-VKG_pdospJ6xliXQ   
7297  Cni2l-VKG_pdospJ6xliXQ   
7298  Cni2l-VKG_pdospJ6xliXQ   
7299  Cni2l-VKG_pdospJ6xliXQ   
7300  Cni2l-VKG_pdospJ6xliXQ   
7301  Cni2l-VKG_pdospJ6xliXQ   
7302  Cni2l-VKG_pdospJ6xliXQ   
7303  Cni2l-VKG_pdospJ6xliXQ   

                                                 review  star   topic 0  \
7135  If I knew I was going to upscale pub to have a...   2.0  0.000000   
7136  I've held off on this review for quite a while...   1.0  0.000000   
7137  Hubert Keller's joint is good... but definitel...   2.0  0.000000   
7138  This was "highly recommended" to us by a coupl...   2.0  0.000000   
7139  OVERALL RATING:\nService: 1\nDecor: 2\nFood: 1...   1.0  0.136302   
7140  Booked the Pyramid Suite. Arrived at 7pm and r...   2.0  0.000000   
7141  Food was actually delicious.  Service and pric...   2.0  0.000000   
7142  American Kobe Burger - Ordered med-well, got i...   2.0  0.000000   
7143  The burger portions are small for the price th...   2.0  0.000000   
7144  Did I really come here and eat a $20 burger? W...   2.0  0.000000   
7145  We had a group lunch here and I thought it wou...   1.0  0.000000   
7146  The one star is for the food that isn't bad - ...   1.0  0.000000   
7147  Unbelievably Slow Service! \n \nPumpkin Martin...   2.0  0.395914   
7148  Pros\nWhipped cream\nButtermilk onion rings\nS...   2.0  0.000000   
7149  Well, I can't speak to the food or service, si...   1.0  0.000000   
7150  Good service and the skinny fries were crispy ...   2.0  0.000000   
7151  I ordered the infamous "ROSSINI" burger and I ...   2.0  0.000000   
7152  We hadn't been to the Burger Bar in quite some...   2.0  0.000000   
7153  Came here looking forward to having the kobe b...   2.0  0.000000   
7154  Went back here recently and the Wagyu Burger w...   2.0  0.000000   
7155  A wonderful place off the strip that you can s...   2.0  0.154858   
7156  For a celebrity chef's burger restaurant it wa...   2.0  0.000000   
7157  Completely disappointed, under seasoned and ov...   2.0  0.000000   
7158  I came here not too long ago based on the Yelp...   2.0  0.000000   
7159  No me gusta. No me gusta at all.\n\nThis shoul...   2.0  0.000000   
7160  While they do offer some of the best burgers I...   2.0  0.000000   
7161  I

In [132]:
count = 1
for text in df_b_all[7135:7304]['review']:
    print()
    print(count,". ", text)
    count += 1


1 .  If I knew I was going to upscale pub to have a "good" burger, I wouldn't have gone here. I went because their website led me to believe that they were a very nice restaurant with off-the-chart mindblowing burgers that would make me say wow. Their burgers were good but not any better than good burgers I've had at many other restaurants. To top it off, 2 burgers, a fry sampler & sauces & 2 beers were $66! Disappointed. Don't be misled by the fancy website stuff.

2 .  I've held off on this review for quite a while, but after strike 3 here I felt compelled to finally write my review. I'm sure this might be hit or miss, but at least this experience has repeated itself.

The concept of the build your burger and of course the Hubert endorsement is a reason that fills the seat, but my love for the Burger Bar unfortunately stops there.

I have gone on 3 separate occasions and the first 2 I have had issues with the burger and recently it was now my wife who had the "Irish Luck" with Lunch

In [74]:
# groups = df_b_all.groupby(['business_id'])

In [75]:
# for name, group in groups:
#     print(name)
#     print(group)

In [76]:
# lets see the topic distribution
# df_stat_all = df_b_all.T.reset_index()

In [77]:
# get all business_ids stored 
bus_id_list = []
for bus_id in df_b_all['business_id']:
    if bus_id not in bus_id_list:
        bus_id_list.append(bus_id)
# stats_df['business_id'] = bus_id_list

In [78]:
cat_list = []
for cat in df_b_all['category']:
    if cat not in cat_list:
        cat_list.append(cat)

In [79]:
name_list = []
for name in df_b_all['name']:
    if name not in name_list:
        name_list.append(name)

In [80]:
# treat each resturant as an indivudal df and store it in a list of df
resturants_df = []
for bus_id in bus_id_list:
    curr_resturant_df = df_b_all.loc[(df_b_all['business_id'] == bus_id)]
    resturants_df.append(curr_resturant_df)
print(len(resturants_df))

100


In [81]:
# get the first resturant from the list
tmp_df = resturants_df[0]
tmp_df;

In [82]:
# calculate the stats of that resturant
stat_sr = tmp_df.mean()
stat_sr;

In [83]:
# iterate over every column with stat detail 
stat_sr_nostar = stat_sr.iloc[1:]
for i in range(len(stat_sr_nostar)):
    print(stat_sr_nostar[i]);

0.02316943567959552
0.018417663832611227
0.01455655277885021
0.007666876678295592
0.10522626261127756
0.05184854949487651
0.018206113830525825
0.009711692704165236
0.012112928851646311
0.035786330699920654
0.004673779803387662
0.013906006186090886
0.021143177127901543
0.005760122486885557
0.024752579233113754
0.014454593802703189
0.006192104137958365
0.0017491576479787526
0.2492584097100065
0.015576394591876801
0.04359948607359795
0.003109494422344451
0.0031241608109880002
0.02882795185445154
0.017184840396363685
0.011126008974586396
0.08368818652122578
0.08100147513633078
0.009774850364378158
0.015807579687618194


In [84]:
cols = ['business_id','name','category', 'topic 0', 'topic 1', 'topic 2',
       'topic 3', 'topic 4', 'topic 5', 'topic 6', 'topic 7', 'topic 8',
       'topic 9', 'topic 10', 'topic 11', 'topic 12', 'topic 13', 'topic 14',
       'topic 15', 'topic 16', 'topic 17', 'topic 18', 'topic 19', 'topic 20',
       'topic 21', 'topic 22', 'topic 23', 'topic 24', 'topic 25', 'topic 26','topic 27',
       'topic 28','topic 29']

In [85]:
series_list = []

for resturant in resturants_df:
    bus_id = resturant['business_id'].iloc[0]
    category = resturant['category'].iloc[0]
    name = resturant['name'].iloc[0]
    stat_sr = resturant.mean()
    stat_sr = stat_sr.iloc[1:] # remove the stars column
    biz_data = [bus_id,name,category] # data for every business comrpising of entire future row values
    for i in range(len(stat_sr)):
        topic_propability = stat_sr[i] * 100 # get the indivudal topic probability 
        biz_data.append(topic_propability)
    tmp_sr = pd.Series(biz_data, cols) # treat each row of the series as a column for the df
    series_list.append(tmp_sr)
    
    

In [86]:
series_list[0]

business_id                               0FUtlsQrJI7LhqDPxLumEw
name                                            Joe's Farm Grill
category       Pizza, Italian, Restaurants, Salad, Local Flav...
topic 0                                                  2.31694
topic 1                                                  1.84177
topic 2                                                  1.45566
topic 3                                                 0.766688
topic 4                                                  10.5226
topic 5                                                  5.18485
topic 6                                                  1.82061
topic 7                                                 0.971169
topic 8                                                  1.21129
topic 9                                                  3.57863
topic 10                                                0.467378
topic 11                                                  1.3906
topic 12                 

In [87]:
stats_df = pd.DataFrame(series_list,columns=cols)
stats_df.append(series_list)
stats_df.head()


business_id              name  \
0  0FUtlsQrJI7LhqDPxLumEw  Joe's Farm Grill   
1  0W4lkclzZThpx3V65bVgig        Schwartz's   
2  2iTsRqUsPGRH1li1WVRvKQ    Carson Kitchen   
3  2weQS-RnoOBhb1KsHKyoSQ        The Buffet   
4  3BCsAgo_1i4xMuTyLKMLRQ        SkinnyFATS   

                                            category   topic 0   topic 1  \
0  Pizza, Italian, Restaurants, Salad, Local Flav...  2.316944  1.841766   
1  Specialty Food, Sandwiches, Food, Imported Foo...  6.247231  1.033486   
2  Bars, Restaurants, Nightlife, Cocktail Bars, A...  5.987994  0.575344   
3  Beauty & Spas, Food, Event Planning & Services...  2.228406  0.603006   
4  Food Delivery Services, Juice Bars & Smoothies...  1.747899  1.018882   

    topic 2   topic 3    topic 4   topic 5   topic 6  ...   topic 20  \
0  1.455655  0.766688  10.522626  5.184855  1.820611  ...   4.359949   
1  1.923833  1.385940   3.381270  2.473048  0.605877  ...  36.874209   
2  0.772781  1.079965  11.297619  4.249134  0.147583  ...   0.322572   
3  1.035680  0.360797  13.055941  1.713159  0.691727  ...   1.412371   
4  0.636233  1.108561  10.035819  1.767835  1.884558  ...   3.087254   

   topic 21  topic 22  topic 23  topic 24  topic 25  topic 26   topic 27  \
0  0.310949  0.312416  2.882795  1.718484  1.112601  8.368819   8.100148   
1  1.265858  0.544051  1.071138  1.232881  0.578504  6.577596   6.556546   
2  0.563241  0.943076  0.776258  1.866582  0.226336  3.100975  34.769070   
3  0.310201  0.270865  0.594542  1.072503  0.137659  4.333153   6.088799   
4  2.096378  0.570430  0.233099  1.502649  0.013229  5.692184  17.713864   

   topic 28  topic 29  
0  0.977485  1.580758  
1  0.640568  0.912432  
2  1.246560  0.405583  
3  1.148005  1.808079  
4  1.388517  0.417956  

[5 rows x 33 columns]

In [95]:
stats_df['category'][0]

'Pizza, Italian, Restaurants, Salad, Local Flavor, American (Traditional)'

In [88]:
stats_df.to_csv('stats100.csv')

In [420]:
# f, ax = plt.subplots(figsize=(10, 8))
# # df_b_all = df_b_all.drop('star',axis = 1)
# corr = df_b_all.corr()
# corr

# # mask = np.zeros_like(corr)
# # mask[np.triu_indices_from(mask)] = True
# # with sns.axes_style("white"):
# #     ax = sns.heatmap(df_b_all, mask=mask, vmax=1, square=True)